In [142]:
import numpy as np
import keras
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
import os
import glob

warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
import data_set_ooc
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 20
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01
FLAGS['patience'] = 3

In [158]:
data = data_set_ooc.DataSet(width = FLAGS['width'], height=FLAGS['height'])

In [159]:
from keras.callbacks import TensorBoard
tb = TensorBoard(log_dir='keras_tb/', histogram_freq=5, write_graph=True, write_images=False)

In [160]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(32, 32, 3)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


In [161]:
model = create_model()

In [162]:
def eval_on_val(data, model, batch_size=20):
    data.reset_val_index()
    n_batches = int((data.num_val_examples / batch_size))
    val_loss = 0
    for batch in range(n_batches):
        (X_batch, y_batch) = data.next_val_batch(batch_size) 
        
        val_loss += model.test_on_batch(X_batch, y_batch)
    return (val_loss / n_batches)
 

In [163]:
n_steps = int(FLAGS['n_epochs'] * (data.num_examples / FLAGS['batch_size']))
print('training for {} steps'.format(n_steps))
X_val, y_val = data.get_validation_set()
best_val_loss = 20
no_val_improvements = 0
for step in range(n_steps):
    X_batch, y_batch = data.next_batch(FLAGS['batch_size'])
    batch_loss = model.train_on_batch(X_batch, y_batch)
    if ((step + 1) % 20 == 0):
        print('loss for batch {}:{}'.format(step, batch_loss))
        
    if ((step + 1) % 100 == 0):
        print('Evaluating on validation set')
        val_loss = eval_on_val(data, model, FLAGS['batch_size'])
        if val_loss < best_val_loss:
            print('Validation improved')
            best_val_loss = val_loss
            no_val_improvements = 0
        else:
            no_val_improvements +=1
            print('Validation has not improved for {} rounds'.format(no_val_improvements))
            if no_val_improvements == FLAGS['patience']:
                print('Validation loss has stopped improving. Stopping training')
                break
        print('validation loss for step {}:{}'.format(step, val_loss))

training for 3776 steps
loss for batch 19:1.7720377445220947
loss for batch 39:1.6299201250076294
loss for batch 59:1.9017510414123535
loss for batch 79:1.6430813074111938
loss for batch 99:1.4196301698684692
Evaluating on validation set
Validation improved
validation loss for step 99:1.6248446474684046
loss for batch 119:1.7226126194000244
loss for batch 139:1.2384369373321533
loss for batch 159:1.7265989780426025
loss for batch 179:1.5586460828781128
loss for batch 199:1.4797132015228271
Evaluating on validation set
Validation improved
validation loss for step 199:1.611971066353169
loss for batch 219:1.2945077419281006
loss for batch 239:1.7211583852767944
loss for batch 259:1.488649606704712
loss for batch 279:1.8336536884307861
loss for batch 299:1.3154619932174683
Evaluating on validation set
Validation improved
validation loss for step 299:1.6031947693926223
loss for batch 319:1.2942259311676025
loss for batch 339:1.5841209888458252
loss for batch 359:1.482114315032959
loss for b

In [109]:
import load_images_ooc
X_test, y_test = load_images_ooc.load_test(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [164]:
preds = model.predict(X_test)

In [165]:
import make_submission
submit = make_submission.makeSubmission(preds, y_test)

In [166]:
submit.tail(20)

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT,image
980,0.480731,0.091640,0.038230,3.138173e-02,0.059931,0.104842,4.453060e-02,0.148713,img_07663.jpg
981,0.489360,0.091023,0.036115,3.233087e-02,0.060564,0.104608,4.333372e-02,0.142666,img_07678.jpg
982,0.425442,0.054473,0.021002,1.168269e-02,0.338403,0.026709,4.236940e-03,0.118051,img_07689.jpg
983,0.487482,0.091121,0.036313,3.263553e-02,0.060848,0.104780,4.390377e-02,0.142918,img_07700.jpg
984,0.861521,0.021484,0.003262,6.886035e-03,0.038153,0.024969,1.936493e-03,0.041788,img_07717.jpg
985,0.544640,0.070526,0.026507,3.604038e-02,0.034945,0.155779,1.828543e-02,0.113277,img_07746.jpg
986,0.921015,0.002207,0.000235,2.029999e-04,0.062197,0.000736,6.619100e-05,0.013341,img_07757.jpg
987,0.266553,0.097018,0.095287,1.118319e-02,0.040244,0.120980,2.957791e-02,0.339157,img_07792.jpg
988,0.675522,0.055076,0.014112,1.870452e-02,0.065001,0.059667,1.585725e-02,0.096060,img_07799.jpg
989,0.511695,0.032804,0.015799,5.942514e-03,0.289963,0.016157,2.782125e-03,0.124856,img_07818.jpg


In [167]:
submit.to_csv('keras_test_ooc_3399_steps.csv', index=False)

In [16]:
callbacks a= [
            EarlyStopping(monitor='val_loss', patience=5, verbose=0),
tb]
hist = model.fit(data.X_train, data.y_train, batch_size=FLAGS['batch_size'], nb_epoch=FLAGS['epochs'],
          shuffle=True, verbose=2, validation_data=(X_val, y_val),
              callbacks=callbacks)



Train on 3021 samples, validate on 756 samples
INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.
INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.
INFO:tensorflow:Summary name convolution2d_2_W:0 is illegal; using convolution2d_2_W_0 instead.
INFO:tensorflow:Summary name convolution2d_2_b:0 is illegal; using convolution2d_2_b_0 instead.
INFO:tensorflow:Summary name convolution2d_3_W:0 is illegal; using convolution2d_3_W_0 instead.
INFO:tensorflow:Summary name convolution2d_3_b:0 is illegal; using convolution2d_3_b_0 instead.
INFO:tensorflow:Summary name convolution2d_4_W:0 is illegal; using convolution2d_4_W_0 instead.
INFO:tensorflow:Summary name convolution2d_4_b:0 is illegal; using convolution2d_4_b_0 instead.
INFO:tensorflow:Summary name dense_1_W:0 is illegal; using dense_1_W_0 instead.
INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.
INFO:tensorflow:Summary n

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [18]:
preds = model.predict(X_test, batch_size=FLAGS['batch_size'], verbose=2)


In [24]:
import make_submission
submit = make_submission.makeSubmission(preds,y_test)

In [27]:
submit.to_csv('keras_32_32.csv', index=False)